In [36]:
import pandas as pd

from mit_d3m.db import get_db
from mit_d3m.utils import restore_dots

db = get_db('ta2')

In [53]:
from piex.explorer import S3PipelineExplorer, MongoPipelineExplorer

piex = S3PipelineExplorer('ml-pipelines-2018')
dbex = MongoPipelineExplorer(db)

In [55]:
sdf2 = dbex.get_pipelines()

In [56]:
sdf2.shape

(2115907, 11)

In [57]:
sdf2.head().T

,0,1,2,3,4
_id,032daa76-954d-44b8-b925-68749386dc66,a65840c8-b455-4190-9eb8-1ab6727dfd2a,b28fc76a-56fa-4eb8-9fcb-8a9b6665c3fc,f81a2bd5-d8e1-4734-af87-e7382a9804af,3b68815e-5344-4ff7-9060-6025de8a838f
dataset,196_autoMpg_dataset_TRAIN,26_radon_seed_dataset_TRAIN,LL0_1027_esl_dataset_TRAIN,196_autoMpg_dataset_TRAIN,LL0_1028_swd_dataset_TRAIN
metric,meanSquaredError,rootMeanSquaredError,meanSquaredError,meanSquaredError,meanSquaredError
name,trivial.median,trivial.median,trivial.median,categorical_encoder/imputer/standard_scaler/xg...,trivial.median
rank,64.3087,0.836823,2.19311,9.07949,0.702826
score,64.3087,0.836823,2.19311,9.07949,0.702826
template,None,None,None,5bd0cf9849e71569e8bf8013,None
test_id,20181024200501872083,20181024200501872083,20181024200501872083,20181024200501872083,20181024200501872083
pipeline,trivial.median,trivial.median,trivial.median,categorical_encoder/imputer/standard_scaler/xg...,trivial.median
data_modality,single_table,single_table,single_table,single_table,single_table


In [41]:
DATA = '../../piex-data/'

In [58]:
def dump_csv(df, collection):
    df.to_csv(os.path.join(DATA, 'csvs', collection + '.csv.gz'), index=False, compression='gzip')

dump_csv(dbex.get_pipelines(), 'pipelines')
dump_csv(dbex.get_tests(), 'tests')
dump_csv(dbex.get_test_results(), 'test_results')

In [23]:
templates = dict()
for template in db.pipelines.find():
    template = restore_dots(template)
    metadata = template['metadata']
    template_name = metadata['name'].replace('/', '.')
    template['id'] = str(template.pop('_id'))
    metadata['insert_ts'] = metadata['insert_ts'].isoformat()
    templates[template_name] = template

In [24]:
list(templates.items())[0]

('single_table.time_series_forecasting.default',
 {'metadata': {'name': 'single_table/time_series_forecasting/default',
   'data_type': 'single_table',
   'task_type': 'time_series_forecasting',
   'insert_ts': '2018-10-23T04:58:04.948000'},
  'primitives': ['featuretools.dfs',
   'sklearn.preprocessing.Imputer',
   'sklearn.preprocessing.StandardScaler',
   'xgboost.XGBRegressor'],
  'hyperparameters': {'xgboost.XGBRegressor#1': {'n_jobs': -1,
    'learning_rate': 0.1,
    'n_estimators': 300,
    'max_depth': 3,
    'gamma': 0,
    'min_child_weight': 1}},
  'init_params': {'featuretools.dfs#1': {'encode': True}},
  'id': '5bceaa5c49e71569e8bf7f7a'})

In [10]:
tests = db.tests.find({'test_id': {'$gte': '20181024200501872083'}})

In [11]:
test_ids = set(test['test_id'] for test in tests)

In [26]:
import json
import os
import gzip

for name, template in templates.items():
    template['id'] = str(template['id'])
    template['metadata']['insert_ts'] = str(template['id'])
    filename = os.path.join(DATA, 'templates', name + '.json.gz')
    with gzip.open(filename, 'wt') as f:
        json.dump(template, f, indent=4)

In [27]:
def dump_pipeline(pipeline):
    pipeline['ts'] = pipeline['ts'].isoformat()
    filename = os.path.join(DATA, 'pipelines', pipeline['id'] + '.json.gz')
    with gzip.open(filename, 'wt') as pipeline_file:
        json.dump(restore_dots(pipeline), pipeline_file)

In [28]:
from tqdm import  tqdm_notebook

In [29]:
query = {}
cursor = db.solutions.find(query)
total = db.solutions.count_documents(query)

In [30]:
total

3558442

In [31]:
for pipeline in tqdm_notebook(cursor, total=total):
    dump_pipeline(pipeline)